# Importing necessary packages and loading the model 

In [4]:
import cv2
import numpy as np

# Load the model
net = cv2.dnn.readNetFromTensorflow("dnn/frozen_inference_graph_coco.pb",
                                    "dnn/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt")

# classnames

In [5]:
# Store Coco Names in a list
classesFile = "coco.names"
classNames = open(classesFile).read().strip().split('\n')
print(classNames)

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'hair brush']


# Image preprocessing

In [6]:
# Load image
img = cv2.imread("car.jpeg")
height, width, _ = img.shape

# Create black image
blank_mask = np.zeros((height, width, 3), np.uint8)

# Create blob from the image
blob = cv2.dnn.blobFromImage(img, swapRB=True)

# Make predictions from the input

In [7]:
# Detect objects
net.setInput(blob)

boxes, masks = net.forward(["detection_out_final", "detection_masks"])
detection_count = boxes.shape[2]


# Post process the output data and Final Result

In [8]:
  for i in range(detection_count):
    # Extract information from detection
    box = boxes[0, 0, i]
    class_id = int(box[1])
    score = box[2]
    
    if score < 0.6:
        continue

    class_name = classNames[class_id]
    
    x = int(box[3] * width)
    y = int(box[4] * height)
    x2 = int(box[5] * width)
    y2 = int(box[6] * height)

    roi = blank_mask[y: y2, x: x2]
    roi_height, roi_width, _ = roi.shape
    # Get the mask
    mask = masks[i, int(class_id)]
    mask = cv2.resize(mask, (roi_width, roi_height))
    _, mask = cv2.threshold(mask, 0.5, 255, cv2.THRESH_BINARY)
    
        # Convert mask to np.uint8
    mask = mask.astype(np.uint8)

    # Apply mask to the region of interest
    masked_img = cv2.bitwise_and(img[y: y2, x: x2], img[y: y2, x: x2], mask=mask)

    # Overlay masked image onto the blank mask
    blank_mask[y: y2, x: x2] = cv2.add(blank_mask[y: y2, x: x2], masked_img)

    # Add label to the segmented image
    cv2.putText(blank_mask, class_name, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

# alpha is the transparency of the first picture
alpha = 0.5
# beta is the transparency of the second picture
beta = 0.5
mask_img = cv2.addWeighted(img, alpha, blank_mask, beta, 0)

cv2.imshow("Masked Image with Labels", mask_img)
cv2.waitKey(0)


-1